# <center>Missão Padrão Delta</center>

## <center><u>``Resumo do código``</u></center>

>### Entra: 
> - 1_Ninjas
> - 2_Delta  
>### Sai: 
> - Long_DataFusion
> - Wide_DataFusion
> - Wide_DataFusion_2022
> - G_Classic
> - G_Classic_Reposto
> - G_Classic_SemReposto
> - G_Rotura
> - G_NumProdutos
---
O objectivo é receber dados dos Ninjas e da Delta e devolver ficheiros prontos para fazer os Dashboards. 


---


In [2]:
import pandas as pd
import numpy as np
import datetime
import json
from ipywidgets import interact

def escrever_excel(dfa, nome):
    dfa.to_excel('D:\\B&N Dados\\Delta\\Padrão\\Padrão_07_2023\\%s.xlsx' %nome, index=False)
    
def escrever_csv(dfa, nome):
    dfa.to_csv('D:\\B&N Dados\\Delta\\Padrão\\Padrão_07_2023\\%s.csv' %nome, index=False)
    
def escrever_txt(dfa, nome):
    dfa.to_csv('D:\\B&N Dados\\Delta\\Padrão\\Padrão_07_2023\\%s.txt' %nome, sep='\t', index=False, header=False)
    
def ler_json(ficheiro):
    with open('D:\\B&N Dados\\Delta\\%s.json' %ficheiro, 'r') as file:
        mapa = json.load(file)
    return mapa

<a id="section0"></a>

# Índice

1. [Data Cleaning](#section1)<br>
2. [DataFusion Long](#section2)<br>
3. [DataFusion Wide](#section3)<br>
4. [Gráficos](#section4)<br>
5. [Escrever](#section5)<br>
6. [Visualizar](#section6)

# O que é preciso substituir:

1. Data Cleaning
   1. [Directória e nº bases Ninjas](#section1.1)
   2. [Directória e nº bases Delta](#section1.2)


---

<a id="section1"></a>
# <center>1. Data Cleaning</center>
[Voltar ao índice](#section0)<br>


In [3]:
''' Ler os dicionários previamente criados para poder usar os códigos de produto e de loja em vez dos nomes '''

mapa_produtos = ler_json("dicionário_produtos")
mapa_produtos = {int(key): value for key, value in mapa_produtos.items()}

mapa_lojas = ler_json("dicionário_lojas")
mapa_lojas = {int(key): value for key, value in mapa_lojas.items()}

mapa_produtos2 = ler_json("dicionário_produtos_23")
mapa_produtos2 = {int(key): value for key, value in mapa_produtos2.items()}   

- # `Ninjas`

> <font color=blue>Ler</font>
<a id="section1.1"></a>

In [4]:
%%time
# Ler o ficheiro long com Stocks e Fornecimento

directória = "D:\\B&N Dados\\Delta\\Padrão\\Padrão_07_2023\\"
num_bases = 1

prefixo = "1_Ninjas"
extensão = ".xlsx" #xlsb


dataframes = []

for i in range(1, num_bases + 1):
    file_path = f"{directória}{prefixo}{i}{extensão}"
    df = pd.read_excel(file_path)
    dataframes.append(df)
    
dfNinjas = pd.concat(dataframes, ignore_index=True)
dfNinjas = dfNinjas.rename(columns={' Data de Resposta': 'DATA', "Código da loja":"STORE", "Nome da Loja":"Loja"})

#dfNinjas = dfNinjas.rename(columns=lambda x: mapa_produtos2.get(x, x))

"""Datetime"""

# se xlsx
dfNinjas['DATA'] = pd.to_datetime(dfNinjas['DATA'], format='%d-%m-%Y') 

# se xlsb
#dfNinjas['DATA'] = pd.to_datetime(dfNinjas['DATA'], unit='d', origin=datetime.datetime(1899, 12, 30)) 

CPU times: total: 500 ms
Wall time: 807 ms


- Lista de produtos e lista de lojas

# <font color=red>É preciso definir melhor o formato do ficheiro dos ninjas para se saber os produtos, preciso de perceber melhor como é gerado na aplicação</font>

In [5]:
# Produtos
ficheiro_Produtos = dfNinjas.columns.tolist()
df_produtos = pd.DataFrame(ficheiro_Produtos[4:], columns=["DESC_ARTIGO"])

# Lojas
df_lojas = pd.DataFrame(dfNinjas["STORE"].unique().tolist(), columns=["STORE"])


'''Colocar em listas para serem chamáveis'''

lojas = df_lojas["STORE"].tolist()
produtos = df_produtos["DESC_ARTIGO"].tolist()
numprod = len(produtos)

> - **Id**

In [6]:
# Extract the day and month components from the 'datetime' column
dfNinjas['day'] = pd.to_datetime(dfNinjas['DATA']).dt.day
dfNinjas['month'] = pd.to_datetime(dfNinjas['DATA']).dt.month

# Create a new column combining day and month
dfNinjas['day_month'] = dfNinjas['day'].astype(str) + '-' + dfNinjas['month'].astype(str)

# Assign unique IDs to each unique day-month combination
dfNinjas['id'], _ = pd.factorize(dfNinjas['day_month'])

dfNinjas = dfNinjas.drop(columns=['day', 'month', 'day_month'])

> - **Vendedor**

In [7]:
# Ler ficheiro
dfVendedor=pd.read_excel("D:\\B&N Dados\\Delta\\Vendedor.xlsx", sheet_name = "Lista Lojas Sonae")
dfVendedor = dfVendedor.rename(columns={"Cód. Loja":"STORE"})

# Criar coluna de reposição
dfNinjas = pd.merge(dfNinjas, dfVendedor[["STORE","Vendedor"]], how="left", on = "STORE")

> - **Reposição**

In [8]:
# Ler o ficheiro com os nomes das lojas que fazem reposição
dfRepos=pd.read_excel("D:\\B&N Dados\\Delta\\Reposição_Sonae_Código.xlsx")

# Criar coluna que determina quais lojas têm reposição
dfNinjas['Reposição'] = [1 if val in dfRepos['STORE'].values else 0 for val in dfNinjas['STORE']]

[Escrever](#section5)

---

<a id="section2.1"></a>
- #  `Delta`

> <font color=blue>Ler</font>
<a id="section1.2"></a>

In [9]:
%%time
# Ler o ficheiro long com Stocks e Fornecimento

directória = "D:\\B&N Dados\\Delta\\Padrão\\Padrão_07_2023\\"
extensão = ".xlsb"
#extensão = ".xlsx"
num_bases = 4

prefixo = "2_Delta"

dataframes = []

for i in range(1, num_bases + 1):
    file_path = f"{directória}{prefixo}{i}{extensão}"
    df = pd.read_excel(file_path)
    dataframes.append(df[['DATA', 'EAN', 'DESC_ARTIGO', 'STORE', "STORE_NAME", 
                         "SOH", "INTRANSIT", "EXPECTED", "PRES_STOCK", "VND (D-1)"]])
    
dfDelta=pd.concat(dataframes, ignore_index=True)

"""Datetime"""

# se xlsx
#dfDelta['DATA'] = pd.to_datetime(dfDelta['DATA'], format='%d-%m-%Y') 

# se xlsb
dfDelta['DATA'] = pd.to_datetime(dfDelta['DATA'], unit='d', origin=datetime.datetime(1899, 12, 30)) 

CPU times: total: 22.4 s
Wall time: 23.2 s


> - **Definir os produtos e lojas em causa**

In [10]:
# Estabelecer o Dataframe mais limitado
dfDelta_Limpo = dfDelta[dfDelta['STORE'].isin(lojas) & dfDelta['DESC_ARTIGO'].isin(produtos)].copy()

# Mudar nomes
dfDelta_Limpo = dfDelta_Limpo.rename(columns={"VND (D-1)": "SELLOUT_1_Dias_Antes", "SOH": "STOCK_1_Dias_Antes"})

> - **Criar colunas de Stock e Sellout para o próprio dia**

In [11]:
# Organizar para atingir os valores relevantes
dfDelta_Limpo = dfDelta_Limpo.sort_values(by=["STORE", "DESC_ARTIGO", "DATA"])

#Agrupar por loja e produto para não haver valores errados 
dfDelta_Limpo["SELLOUT"] = dfDelta_Limpo.groupby(["STORE", "DESC_ARTIGO"])["SELLOUT_1_Dias_Antes"].shift(-1)
dfDelta_Limpo["STOCK"] = dfDelta_Limpo.groupby(["STORE", "DESC_ARTIGO"])["STOCK_1_Dias_Antes"].shift(-1)

> - **Retirar os dias que não têm interesse (último dia)**

In [12]:
# O dia final foi utilizado para conseguir dados do dia para stocks e sellouts mas não é dia de missão
segunda = dfDelta_Limpo['DATA'].unique()

dfDelta_Limpo = dfDelta_Limpo.loc[dfDelta_Limpo['DATA'] < segunda[-1]]
dfDelta_Limpo_Org = dfDelta_Limpo.iloc[:, np.r_[:5,11,6:9,10]]

## <center><font color=orange>Output DataCleaning

In [13]:
dfNinjas = dfNinjas.iloc[:, np.r_[5+numprod,0:5+numprod,6+numprod]].copy()
dfDelta = dfDelta_Limpo_Org.copy()

---

<a id="section2"></a>
# <center> 2. DataFusion Long (1)</center>
[Voltar ao índice](#section0)

> - __dfNinjas__ já definido
> - __dfDelta__ já definido
> - __produtos__ já definidos

- #  `dfFinal1`

In [14]:
resto = dfNinjas.columns.difference(produtos)
# Passar ficheiro ninjas para long
dfNinjasLong = dfNinjas.melt(id_vars=resto, value_vars=produtos, var_name='DESC_ARTIGO', value_name='NinjaInfo')


#Mergir
dfMeio = pd.merge(dfNinjasLong, dfDelta, how="left", on = ["DATA","STORE", "DESC_ARTIGO"]) 
#dfMeio = dfMeio.drop(columns=["Loja"])

dfFinal1=dfMeio.copy()

> STK

In [15]:
# Definir coluna de STK (soma de stocks com stock em trânsito)

dfFinal1["STK"] = dfFinal1["STOCK"] + dfFinal1["INTRANSIT"] + dfFinal1["EXPECTED"]

> Balanço

In [16]:
# Definir coluna de Balanço (razão entre sellout e stock total)

dfFinal1["Balanço"] = dfFinal1["SELLOUT"] / dfFinal1["STK"]

> Rotura

In [17]:
# Definir coluna de rotura (se stock menor ou igual a 0 e existe Linear)

dfFinal1["ROTURA"] = np.where((dfFinal1["STOCK"] <= 0) & (dfFinal1["PRES_STOCK"] > 0), 1, 0)

>Sinal

In [18]:
# Categorias possíveis
mapping = {1: "Presente com Stock",
           2: "Ausente com Stock",
           3: "Presente sem Stock",
           4: "Ausente sem Stock",
           5: "Presente sem Registo",
           6: "Ausente sem Registo"}

# Definir coluna de sinal
dfFinal1["Sinal"] = np.where((dfFinal1["STOCK"] > 0) & (dfFinal1["NinjaInfo"] == 1), 1,
                     np.where((dfFinal1["STOCK"] > 0) & (dfFinal1["NinjaInfo"] == 0), 2,
                     np.where((dfFinal1["STOCK"] <= 0) & (dfFinal1["NinjaInfo"] == 1), 3,
                     np.where((dfFinal1["STOCK"] <= 0) & (dfFinal1["NinjaInfo"] == 0), 4,
                     np.where((dfFinal1["STOCK"].isna()) & (dfFinal1["NinjaInfo"] == 1), 5,
                     np.where((dfFinal1["STOCK"].isna()) & (dfFinal1["NinjaInfo"] == 0), 6,
                     np.nan))))))

# Substituir números pelas expressões escolhidas
dfFinal1["Sinal"] = dfFinal1["Sinal"].map(mapping)

> Ciclos

In [19]:
dfFinal1["Ciclos"]=dfFinal1["STOCK"]/dfFinal1["PRES_STOCK"]

> Adequação

In [20]:
dfFinal1["Adequação"] = np.where(
                        dfFinal1["Ciclos"] > 1.1,
                        "Stock Suficiente",
                       np.where(
                        (dfFinal1["Ciclos"] <= 1.1) & (dfFinal1["INTRANSIT"] + dfFinal1["EXPECTED"] + dfFinal1["STOCK"] >= dfFinal1["PRES_STOCK"]),
                        "Stock Insuf c Forn Adequado",
                       np.where(
                        (dfFinal1["Ciclos"] <= 1.1) & (dfFinal1["INTRANSIT"] + dfFinal1["EXPECTED"] + dfFinal1["STOCK"] < dfFinal1["PRES_STOCK"]),
                        "Stock Insuf c Forn Desadequado",
                        "")))

> Dias para a rotura de stock

In [21]:
dfFinal1["Dias_para_Rotura"] = dfFinal1["STOCK"] / dfFinal1["SELLOUT"]

> Dias para a rotura de prateleira

In [22]:
dfFinal1["Dias_para_Rotura_Linear"] = dfFinal1["PRES_STOCK"] / dfFinal1["SELLOUT"]

> Roturas de prateleira (meh)

- ###  `Organizar`

## <center><font color=orange>Output DataFusion Long

In [90]:
#df1 = dfFinal1.iloc[:, np.r_[5,0:2,7:16,2:5,6,16:19,19:24]].copy()  # Sem hora
df1 = dfFinal1.iloc[:, np.r_[5,0:3,7:16,3:5,6,16:19,19:24]].copy()   # Com hora

In [92]:
[f"{index}: {column}" for index, column in enumerate(df1)]

['0: Vendedor',
 '1: DATA',
 '2: Hora',
 '3: Loja',
 '4: DESC_ARTIGO',
 '5: NinjaInfo',
 '6: EAN',
 '7: STORE_NAME',
 '8: STOCK',
 '9: INTRANSIT',
 '10: EXPECTED',
 '11: PRES_STOCK',
 '12: SELLOUT',
 '13: Reposição',
 '14: STORE',
 '15: id',
 '16: STK',
 '17: Balanço',
 '18: ROTURA',
 '19: Sinal',
 '20: Ciclos',
 '21: Adequação',
 '22: Dias_para_Rotura',
 '23: Dias_para_Rotura_Linear']

- <font color = red>Se houver lapsos a nível de Cardex:

In [91]:
df1 = df1.dropna(subset=["STOCK"])

- Meter aquelas métricas

In [93]:
dfJuntar = pd.read_csv('D:\\B&N Dados\\Delta\\Stocks\\StocksTotal\\Mil_MétricasCerto.csv')

In [94]:
dfJuntar.columns.tolist()

['DATA',
 'EAN',
 'DESC_ARTIGO',
 'STORE',
 'STORE_NAME',
 'INTRANSIT',
 'EXPECTED',
 'PRES_STOCK',
 'STOCK',
 'STOCK_1_Dias_Antes',
 'SELLOUT',
 'SELLOUT_1_Dias_Antes',
 'Prophet',
 'XGBoost',
 'ROTURA',
 'PRE_ROTURA',
 'Volatilidade_30',
 'Volatilidade_60',
 'Volatilidade_120',
 'Volatilidade_180',
 'Percentagem_Roturas_30',
 'Percentagem_Roturas_60',
 'Percentagem_Roturas_120',
 'Percentagem_Roturas_180',
 'New_Supply',
 'Percentagem_Supply_30',
 'Percentagem_Supply_60',
 'Percentagem_Supply_120',
 'Percentagem_Supply_180',
 'SELLOUT_fds',
 'SELLOUT_semana',
 'Efeito_Fds_30',
 'Efeito_Fds_60',
 'Efeito_Fds_120',
 'Efeito_Fds_180',
 'InterSupply',
 'InterSupplyMed_30',
 'InterSupplyMed_60',
 'InterSupplyMed_120',
 'InterSupplyMed_180',
 'Tempo_Indisponível',
 'Tempo_Indisponível_30',
 'Tempo_Indisponível_60',
 'Tempo_Indisponível_120',
 'Tempo_Indisponível_180',
 'Stock_Borderline',
 'Tempo_Stock_Borderline_30',
 'Tempo_Stock_Borderline_60',
 'Tempo_Stock_Borderline_120',
 'Tempo_Sto

In [76]:
dfJuntar=dfJuntar[dfJuntar.DATA>='2023-05-05'][['DATA','STORE', "DESC_ARTIGO",'Volatilidade_30',
 'Volatilidade_60',
 'Volatilidade_120',
 'Volatilidade_180',
 'Percentagem_Roturas_30',
 'Percentagem_Roturas_60',
 'Percentagem_Roturas_120',
 'Percentagem_Roturas_180',
 'New_Supply',
 'Percentagem_Supply_30',
 'Percentagem_Supply_60',
 'Percentagem_Supply_120',
 'Percentagem_Supply_180',
 'SELLOUT_fds',
 'SELLOUT_semana',
 'Efeito_Fds_30',
 'Efeito_Fds_60',
 'Efeito_Fds_120',
 'Efeito_Fds_180',
 'InterSupply',
 'InterSupplyMed_30',
 'InterSupplyMed_60',
 'InterSupplyMed_120',
 'InterSupplyMed_180',
 'Tempo_Indisponível',
 'Tempo_Indisponível_30',
 'Tempo_Indisponível_60',
 'Tempo_Indisponível_120',
 'Tempo_Indisponível_180',
 'Stock_Borderline',
 'Tempo_Stock_Borderline_30',
 'Tempo_Stock_Borderline_60',
 'Tempo_Stock_Borderline_120',
 'Tempo_Stock_Borderline_180',
 'Linear_Incompleto',
 'Tempo_Linear_Incompleto_30',
 'Tempo_Linear_Incompleto_60',
 'Tempo_Linear_Incompleto_120',
 'Tempo_Linear_Incompleto_180',
 'Sem_Vendas',
 'Sem_Vendas_30',
 'Sem_Vendas_60',
 'Sem_Vendas_120',
 'Sem_Vendas_180',
 'ROTURA_fds',
 'ROTURA_semana',
 'Vendas_Perdidas_30',
 'Vendas_Perdidas_60',
 'Vendas_Perdidas_120',
 'Vendas_Perdidas_180']]

In [84]:
len(dfJuntar.columns)

53

In [78]:
# Extract the day and month components from the 'datetime' column
dfJuntar['day'] = pd.to_datetime(dfJuntar['DATA']).dt.day
dfJuntar['month'] = pd.to_datetime(dfJuntar['DATA']).dt.month

# Create a new column combining day and month
dfJuntar['day_month'] = dfJuntar['day'].astype(str) + '-' + dfJuntar['month'].astype(str)

# Assign unique IDs to each unique day-month combination
dfJuntar['id'], _ = pd.factorize(dfJuntar['day_month'])

dfJuntar = dfJuntar.drop(columns=['day', 'month', 'day_month', 'DATA'])

In [85]:
df1 = pd.merge(df1, dfJuntar, on = ["id","STORE", "DESC_ARTIGO"], how = "left")

In [87]:
df1.head()

,Vendedor,DATA,Hora,Loja,DESC_ARTIGO,NinjaInfo,EAN,STORE_NAME,STOCK,INTRANSIT,...,Sem_Vendas_30,Sem_Vendas_60,Sem_Vendas_120,Sem_Vendas_180,ROTURA_fds,ROTURA_semana,Vendas_Perdidas_30,Vendas_Perdidas_60,Vendas_Perdidas_120,Vendas_Perdidas_180
0,Brizida Almeida,2023-07-23,15:03:00,Continente Amadora,CAFÉ DELTA Q QALIDUS 40CAP,1.0,5.601082e+12,CNT AMADORA,57.0,0.0,...,6.666667,8.333333,4.166667,4.444444,0.0,NaN,0.0,0.0,0.0,0.0
1,Carla Teixeira,2023-07-23,19:37:00,Continente Antas,CAFÉ DELTA Q QALIDUS 40CAP,1.0,5.601082e+12,CNT ANTAS,168.0,0.0,...,23.333333,33.333333,29.166667,30.555556,0.0,NaN,0.0,0.0,0.0,0.0
2,Carla Teixeira,2023-07-22,21:47:00,Continente Arrábida (Gaia Jardim),CAFÉ DELTA Q QALIDUS 40CAP,1.0,5.601082e+12,CNT ARRABIDA,18.0,0.0,...,50.000000,38.333333,30.833333,31.111111,0.0,NaN,0.0,0.0,0.0,0.0
3,Andreia Gonçalves,2023-07-23,12:49:00,Continente Beja,CAFÉ DELTA Q QALIDUS 40CAP,1.0,5.601082e+12,CNT BEJA,109.0,0.0,...,3.333333,3.333333,2.500000,10.555556,0.0,NaN,0.0,0.0,0.0,0.0
4,Susana Gomes,2023-07-21,11:41:00,Continente Braga,CAFÉ DELTA Q QALIDUS 40CAP,1.0,5.601082e+12,CNT BRAGA,42.0,0.0,...,23.333333,26.666667,21.666667,23.333333,0.0,NaN,0.0,0.0,0.0,0.0


[Escrever](#section5) <br>
[Visualizar](#section6)

<a id="section3"></a>
# <center>3. DataFusion Wide (2)</center>
[Voltar ao índice](#section0)<br>

> - __dfNinjas__ já definido
> - __dfDelta__ já definido
> - __produtos__ já definidos

- #  `dfFinal2`

In [471]:
# Dataframe mergido no fim
dfMeio2 = dfNinjas.copy()


for coluna in produtos:       
    
    #Cada dfStocks é um dataframe com o produto específico
    dfStocks=dfDelta[dfDelta["DESC_ARTIGO"]==coluna][["DATA", "STORE", "STOCK", "PRES_STOCK", "INTRANSIT", "EXPECTED", "SELLOUT"]]   #Dados Ninjas, seleccionar colunas do café específico
 
    #Juntar dados de acordo com o dia e a loja
    dfMeio2=pd.merge(dfMeio2, dfStocks, how="left", on = ["DATA","STORE"])  
    
    #Mudar nomes para ser adaptado a cada produto
    dfMeio2=dfMeio2.rename(columns={"STOCK": "STOCK %s" % coluna, "PRES_STOCK":"PRESLINEAR %s" % coluna, "INTRANSIT":"INTRANSIT %s" % coluna, "EXPECTED":"EXPECTED %s" % coluna, "SELLOUT":"SELLOUT %s" % coluna})                       #Nomear coluna nova
    
dfFinal2=dfMeio2.copy()

---

> STK

In [472]:
for i in produtos:

    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "STOCK %s" % i
    
    stk = "STK %s" %i
      
    dfFinal2[stk] = dfFinal2[stock] + dfFinal2[esperado] + dfFinal2[transito]

> Balanço

In [473]:
for i in produtos:
    stk = "STK %s" %i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" %i
    
    balance = "BALANCE %s" %i
    
    dfFinal2[balance] = dfFinal2[sellout] / dfFinal2[stk]
    
    

> Rotura

In [474]:
for i in produtos:
    rotura  = "ROTURA %s" % i
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    
    dfFinal2[rotura] = np.where((dfFinal2[stock] > 0) & ~(dfFinal2[i].isna()), 0, 
                      np.where((dfFinal2[stock] <= 0) & (dfFinal2[preslinear] > 0) & ~(dfFinal2[i].isna()), 1, 
                      np.nan))


> Pré-Rotura

In [475]:
for i in produtos:
    stock  = "STOCK %s" % i
    preslinear = "PRESLINEAR %s" % i
    pre_rotura= "PRÉ-ROTURA %s" % i
    
    dfFinal2[pre_rotura] = (dfFinal2[stock] < dfFinal2[preslinear]).astype(int)

>Sinal

In [476]:
#dfFinal2["Sinal"]= pd.Series(dtype='int')

# Define the mappings of numbers to strings
mapping = {1: "Presente com Stock",
           2: "Ausente com Stock",
           3: "Presente sem Stock",
           4: "Ausente sem Stock",
           5: "Presente sem Registo",
           6: "Ausente sem Registo"}

for i in produtos:
    sinal = "SINAL %s" % i
    stock  = "STOCK %s" % i
    
    dfFinal2[sinal] = np.where((dfFinal2[stock] > 0) & (dfFinal2[i] == 1), 1,
                     np.where((dfFinal2[stock] > 0) & (dfFinal2[i] == 0), 2,
                     np.where((dfFinal2[stock] <= 0) & (dfFinal2[i] == 1), 3,
                     np.where((dfFinal2[stock] <= 0) & (dfFinal2[i] == 0), 4,
                     np.where((dfFinal2[stock].isna()) & (dfFinal2[i] == 1), 5,
                     np.where((dfFinal2[stock].isna()) & (dfFinal2[i] == 0), 6,
                     np.nan))))))


    # Map the numbers to the corresponding strings
    dfFinal2[sinal] = dfFinal2[sinal].map(mapping)
    

# 1: Há Stock e Sinal Positivo Ninja "Verdadeiramente Correcto"
# 2: Há Stock e Sinal Negativo Ninja "Falso Positivo"
# 3: Não há Stock e Sinal Positivo Ninja "Falso Negativo"
# 4: Não há Stock e Sinal Negativo Ninja "Rejeição Correcta"

> Ciclos

In [477]:
for i in produtos:
    ciclos = "CICLO %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    dfFinal2[ciclos]=dfFinal2[stock]/dfFinal2[linear]

> Adequação

In [478]:
dfFinal2 = dfFinal2.copy()
# Define the mappings of numbers to strings
mapping = {1: "Stock Suficiente",
           2: "Stock Insuf c Forn Adequado",
           3: "Stock Insuf c Forn Desadequado"}

for i in produtos:
    adequa = "ADEQUAÇÃO %s" % i
    ciclos = "CICLO %s" % i
    transito = "INTRANSIT %s" % i
    esperado = "EXPECTED %s" % i
    stock  = "STOCK %s" % i
    linear = "PRESLINEAR %s" % i
    
    dfFinal2[adequa] = np.where(dfFinal2[ciclos] > 1.1, 1,
                      np.where((dfFinal2[ciclos] <= 1.1) & (dfFinal2[transito] + dfFinal2[esperado] + dfFinal2[stock] >= dfFinal2[linear]), 2,
                      np.where((dfFinal2[ciclos] <= 1.1) & (dfFinal2[transito] + dfFinal2[esperado] + dfFinal2[stock] < dfFinal2[linear]), 3, 
                      np.nan)))
    
    # Map the numbers to the corresponding strings
    dfFinal2[adequa] = dfFinal2[adequa].map(mapping)

- Dias para a rotura

In [479]:
for i in produtos:
    stock  = "STOCK %s" % i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" % i
    dias_rotura = "Dias_para_Rotura %s" % i
    
    dfFinal2[dias_rotura] = dfFinal2[stock]/dfFinal2[sellout]

- Dias para a rotura de prateleira

In [480]:
for i in produtos:
    linear = "PRESLINEAR %s" % i
    #msa = "MSA %s" %i
    sellout = "SELLOUT %s" % i
    dias_rotura_prat = "Dias_para_Rotura_Linear %s" % i
    
    dfFinal2[dias_rotura_prat] = dfFinal2[linear] / dfFinal2[sellout]

---

# `2022`

In [481]:
# Ler o ficheiro long com Stocks e Fornecimento
dfDelta_2022=pd.read_excel("D:\\B&N Dados\\Delta\\Padrão\\Padrão_07_2023\\2_Delta_2022Id.xlsx", sheet_name="Sheet1")

#### dfFinal2_2022

In [482]:
# Dataframe mergido no fim
dfNinjas_2022=dfNinjas.copy()


for coluna in produtos:       
    
    #Cada dfStocks é um dataframe com o produto específico
    dfStocks=dfDelta_2022[dfDelta_2022["DESC_ARTIGO"]==coluna][["id", "STORE", "STOCK", "PRES_STOCK", "INTRANSIT", "EXPECTED", "SELLOUT"]]   #Dados Ninjas, seleccionar colunas do café específico
    
    #Juntar dados de acordo com o dia e a loja
    dfNinjas_2022=pd.merge(dfNinjas_2022, dfStocks, how="left", on = ["id","STORE"])  
    
    #Mudar nomes para ser adaptado a cada produto
    dfNinjas_2022=dfNinjas_2022.rename(columns={"STOCK": "STOCK22 %s" % coluna, "PRES_STOCK":"PRESLINEAR22 %s" % coluna, "INTRANSIT":"INTRANSIT22 %s" % coluna, "EXPECTED":"EXPECTED22 %s" % coluna, "SELLOUT":"SELLOUT22 %s" % coluna})                       #Nomear coluna nova

dfFinal2_2022 = dfNinjas_2022

## <center><font color=orange>Output DataFusion Wide

In [483]:
df2 = dfFinal2.copy()
df2_2022 = dfFinal2_2022.copy()

- <font color = red>Se houver lapsos a nível de Cardex:

In [484]:
for i in produtos:

    stock  = "STOCK %s" % i
    stock22  = "STOCK22 %s" % i
    
    df2.loc[df2[stock].isna(), i] = np.nan
    df2_2022.loc[df2_2022[stock22].isna(), i] = np.nan

[Escrever](#section5) <br>
[Visualizar](#section6)

---

<a id="section4"></a>
# <center>4. Gráficos (3)</center>
[Voltar ao índice](#section0)<br>

> - __produtos__ já definidos

---

- #  `dfFinal3`

In [485]:
dfFinal3 = df2.copy()

# Tabelas

#  <span style="color:green"><u>Classic</u>:</span>  Proporção de presença por produto

In [486]:
# Média tirando os missing values
df_Med = dfFinal3[produtos].mean(numeric_only=True)

# Passar para dataframe
df_Classic = df_Med.to_frame()

# Mudanças para ficar legível
df_Classic = df_Classic.reset_index()
df_Classic = df_Classic.rename(columns={'index': 'Produto', 0: 'Frequência'})

- ###  <span style="color:BLUE">Com e sem Reposição</span>

In [487]:
# Definir se os produtos têm reposição

# Sim
df_ComRep = dfFinal3[dfFinal3["Reposição"] == 1]

# Não
df_SemRep = dfFinal3[dfFinal3["Reposição"] == 0]

>> <font color=blue>Com</font>

In [488]:
# Proporção
serie_MedRep = df_ComRep[produtos].mean(numeric_only=True)

# Para df
df_Classic_ComRepo = serie_MedRep.to_frame()

# Mudanças para ficar legível
df_Classic_ComRepo = df_Classic_ComRepo.reset_index()
df_Classic_ComRepo = df_Classic_ComRepo.rename(columns={'index': 'Produto', 0: 'Frequência'})

>> <font color=blue>Sem

In [489]:
# Proporção
serie_MedSemRep = df_SemRep[produtos].mean(numeric_only=True)

# Proporção
df_Classic_SemRepo = serie_MedSemRep.to_frame()

# Mudanças para ficar legível
df_Classic_SemRepo = df_Classic_SemRepo.reset_index()
df_Classic_SemRepo = df_Classic_SemRepo.rename(columns={'index': 'Produto', 0: 'Frequência'})

---

# <font color=green> <u>Final</u>:</font> Proporção de presença por loja

> - **Número de produtos presentes**

In [490]:
# Criar coluna com o número dos produtos em causa presentes no instante

dfFinal3["Num_Produtos"] = dfFinal3[produtos].sum(axis=1)

dfFinal3.loc[(dfFinal3["Num_Produtos"] == 0) & (dfFinal3[produtos[1]].isna()), "Num_Produtos"] = np.nan

In [491]:
# com reposição
dfFinal3["Num_ProdutosRep"] = np.nan
dfFinal3.loc[dfFinal3['Reposição'] == 1, 'Num_ProdutosRep'] = dfFinal3["Num_Produtos"]


# sem reposição
dfFinal3["Num_ProdutosNRep"] = np.nan
dfFinal3.loc[dfFinal3['Reposição'] == 0, 'Num_ProdutosNRep'] = dfFinal3["Num_Produtos"]

# Coluna para ter o número de produtos contratados pela marca
dfFinal3["Produtos_Contratados"] = dfFinal3[produtos].notna().sum(axis=1)

In [492]:
'''Fazer colunas de proporção de produtos presentes em loja, depois especificando para quando têm e não têm reposição'''

df_NumProd = dfFinal3[["Loja","Num_Produtos","Num_ProdutosRep","Num_ProdutosNRep"]].copy()
df_NumProd["Prop_Produtos"] = (df_NumProd["Num_Produtos"] / dfFinal3["Produtos_Contratados"])*100
df_NumProd["Prop_ProdutosRep"] = (df_NumProd["Num_ProdutosRep"] / dfFinal3["Produtos_Contratados"])*100
df_NumProd["Prop_ProdutosNRep"] = (df_NumProd["Num_ProdutosNRep"] / dfFinal3["Produtos_Contratados"])*100

In [493]:
# Acrescentar à base original
#dfNinjas["Prop_Produtos"] = df_NumProd["Prop_Produtos"]
dfFinal3["Prop_Produtos"] = df_NumProd["Prop_Produtos"]

---

# <font color=green><u> Roturas</u>:</font> Proporção de roturas por produto

In [494]:
df_RotMed = dfFinal3[["ROTURA %s" % produto for produto in produtos]].mean(numeric_only=True)

# Passar para dataframe
df_Rot = df_RotMed.to_frame()
df_Rot = df_Rot.reset_index()
df_Rot = df_Rot.rename(columns={'index': 'Produto', 0: 'Frequência'})
df_Rot["Frequência"] = df_Rot["Frequência"]*100

## <center><font color=orange>Output DataFusion Wide

In [495]:
df2_2 = dfFinal3

#df_Classic
#df_Classic_ComRepo
#df_Classic_SemRepo
#df_Rot
#df_NumProd

[Visualizar](#section6)

---

<a id="section5"></a>
# <center><u>Escrever</u></center>
[Voltar ao índice](#section0)

### <center><font color=green>DataCleaning</font></center>

In [496]:
escrever_txt(df_produtos, "produtos")
escrever_txt(df_lojas, "lojas")

### <center><font color=green>DataFusion Long</font></center>

In [73]:
escrever_excel(df1, "Long_DataFusion_MétricasNovas")

In [72]:
df1.head()

,Vendedor,DATA,Hora,Loja,DESC_ARTIGO,NinjaInfo,EAN,STORE_NAME,STOCK,INTRANSIT,...,Sem_Vendas_30_y,Sem_Vendas_60_y,Sem_Vendas_120_y,Sem_Vendas_180_y,ROTURA_fds_y,ROTURA_semana_y,Vendas_Perdidas_30_y,Vendas_Perdidas_60_y,Vendas_Perdidas_120_y,Vendas_Perdidas_180_y
0,Brizida Almeida,2023-07-23,15:03:00,Continente Amadora,CAFÉ DELTA Q QALIDUS 40CAP,1.0,5.601082e+12,CNT AMADORA,57.0,0.0,...,6.666667,8.333333,4.166667,4.444444,0.0,NaN,0.0,0.0,0.0,0.0
1,Carla Teixeira,2023-07-23,19:37:00,Continente Antas,CAFÉ DELTA Q QALIDUS 40CAP,1.0,5.601082e+12,CNT ANTAS,168.0,0.0,...,23.333333,33.333333,29.166667,30.555556,0.0,NaN,0.0,0.0,0.0,0.0
2,Carla Teixeira,2023-07-22,21:47:00,Continente Arrábida (Gaia Jardim),CAFÉ DELTA Q QALIDUS 40CAP,1.0,5.601082e+12,CNT ARRABIDA,18.0,0.0,...,50.000000,38.333333,30.833333,31.111111,0.0,NaN,0.0,0.0,0.0,0.0
3,Andreia Gonçalves,2023-07-23,12:49:00,Continente Beja,CAFÉ DELTA Q QALIDUS 40CAP,1.0,5.601082e+12,CNT BEJA,109.0,0.0,...,3.333333,3.333333,2.500000,10.555556,0.0,NaN,0.0,0.0,0.0,0.0
4,Susana Gomes,2023-07-21,11:41:00,Continente Braga,CAFÉ DELTA Q QALIDUS 40CAP,1.0,5.601082e+12,CNT BRAGA,42.0,0.0,...,23.333333,26.666667,21.666667,23.333333,0.0,NaN,0.0,0.0,0.0,0.0


### <center><font color=green>DataFusion Wide</font></center>

In [498]:
#escrever_excel(df2_1, "Wide_DataFusion_Inc")

escrever_excel(df2_2, "Wide_DataFusion")
escrever_excel(df2_2022, "Wide_DataFusion2022")

In [499]:
df2_2[df2_2["Prop_Produtos"]>100]

,Vendedor,Loja,STORE,DATA,Hora,CAFÉ DELTA Q QALIDUS 40CAP,CAFÉ DELTA Q QHARACTER 40CAP,CAFÉ GINGA 10CAP,CAFÉ BELLISSIMO AROMA INTENSO 30CAP,CAFÉ SOLÚVEL DELTA FRASCO 200G,...,Dias_para_Rotura_Linear CAFÉ GINGA 10CAP,Dias_para_Rotura_Linear CAFÉ BELLISSIMO AROMA INTENSO 30CAP,Dias_para_Rotura_Linear CAFÉ SOLÚVEL DELTA FRASCO 200G,Dias_para_Rotura_Linear CAFÉ DELTA SOLÚVEL CREME 80 GR,Dias_para_Rotura_Linear BEBIDA CEREAIS DELTA C/20%CAFE FR 200G,Num_Produtos,Num_ProdutosRep,Num_ProdutosNRep,Produtos_Contratados,Prop_Produtos


In [508]:
[f"{index}: {column}" for index, column in enumerate(df2_2)]

['0: Vendedor',
 '1: Loja',
 '2: STORE',
 '3: DATA',
 '4: Hora',
 '5: CAFÉ DELTA Q QALIDUS 40CAP',
 '6: CAFÉ DELTA Q QHARACTER 40CAP',
 '7: CAFÉ GINGA 10CAP',
 '8: CAFÉ BELLISSIMO AROMA INTENSO 30CAP',
 '9: CAFÉ SOLÚVEL DELTA FRASCO 200G',
 '10: CAFÉ DELTA SOLÚVEL CREME 80 GR',
 '11: BEBIDA CEREAIS DELTA C/20%CAFE FR 200G',
 '12: id',
 '13: Reposição',
 '14: STOCK CAFÉ DELTA Q QALIDUS 40CAP',
 '15: PRESLINEAR CAFÉ DELTA Q QALIDUS 40CAP',
 '16: INTRANSIT CAFÉ DELTA Q QALIDUS 40CAP',
 '17: EXPECTED CAFÉ DELTA Q QALIDUS 40CAP',
 '18: SELLOUT CAFÉ DELTA Q QALIDUS 40CAP',
 '19: STOCK CAFÉ DELTA Q QHARACTER 40CAP',
 '20: PRESLINEAR CAFÉ DELTA Q QHARACTER 40CAP',
 '21: INTRANSIT CAFÉ DELTA Q QHARACTER 40CAP',
 '22: EXPECTED CAFÉ DELTA Q QHARACTER 40CAP',
 '23: SELLOUT CAFÉ DELTA Q QHARACTER 40CAP',
 '24: STOCK CAFÉ GINGA 10CAP',
 '25: PRESLINEAR CAFÉ GINGA 10CAP',
 '26: INTRANSIT CAFÉ GINGA 10CAP',
 '27: EXPECTED CAFÉ GINGA 10CAP',
 '28: SELLOUT CAFÉ GINGA 10CAP',
 '29: STOCK CAFÉ BELLISSIMO 

### <center><font color=green>Gráficos</font></center>


- Classic

In [500]:
# Normal
escrever_excel(df_Classic, "G_Classic")

# Com Reposição
escrever_excel(df_Classic_ComRepo, "G_Classic_Reposto")

# Sem Reposição
escrever_excel(df_Classic_SemRepo, "G_Classic_SemReposto")

- Roturas

In [501]:
escrever_excel(df_Rot, "G_Rotura")

- Número de produtos

In [502]:
escrever_excel(df_NumProd, "G_NumProdutos")

<a id="section6"></a>
# <center><u>Visualizar</u></center>
[Voltar ao índice](#section0)

- # `Long_DataFusion`

> Filtrar produto

> Filtrar loja

- # `Wide_DataFusion`

[Voltar ao índice](#section0)